# How to Analyse Large Datasets on Your Laptop.

## Memory

In [1]:
import pandas as pd
import numpy as np
import joblib

from multiprocessing import Pool, cpu_count
import pandas as pd
import numpy as np
import timeit
import time
%load_ext line_profiler
%load_ext cython

/usr/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


you can download data from here- https://data.world/dataquest/mlb-game-logs

In [2]:
gl = pd.read_csv('https://query.data.world/s/cfheaoqgg2wfygjc6r5ebdfvgpemp5', low_memory=False)
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [3]:
gl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 211.2+ MB


In [4]:
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 861.6 MB


<img src="files/df_blocks.png">
https://www.dataquest.io/blog/pandas-big-data/

### Numbers

In [5]:
for dtype in ['float','int','object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 1.29 MB
Average memory usage for int columns: 1.12 MB
Average memory usage for object columns: 9.53 MB


In [6]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

7.87 MB
1.48 MB


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [7]:
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

100.99 MB
50.49 MB


,before,after
float32,NaN,77.0
float64,77.0,NaN


### String

<img src="numpy_vs_python.png">
https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/

In [8]:
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

converted_obj = pd.DataFrame()

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [9]:
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.astype(str).apply(pd.Series.value_counts)

752.74 MB
51.66 MB


,before,after
category,NaN,78.0
object,78.0,NaN


In [10]:
optimized_gl = gl.copy()

optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

print(mem_usage(gl))
print(mem_usage(optimized_gl))

optimized_gl[converted_obj.columns] = converted_obj

mem_usage(optimized_gl)

861.60 MB
804.71 MB


'103.63 MB'

In [11]:
display(optimized_gl[['date']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Data columns (total 1 columns):
date    171907 non-null uint32
dtypes: uint32(1)
memory usage: 671.6 KB


None

In [12]:
optimized_gl['date'] = pd.to_datetime(optimized_gl.date.values,format='%Y%m%d')

print(mem_usage(optimized_gl))

104.28 MB


### Conclusions:
* Optimize  types of number variables
* This quite usefull to store text variables as categorical

## How to save

In [13]:
optimized_gl.to_csv('optim_gl.csv', index=False)

In [14]:
joblib.dump(optimized_gl, 'optim_gl.jbl')

['optim_gl.jbl']

In [15]:
%timeit gl_from_csv =pd.read_csv('optim_gl.csv', low_memory=False)
%timeit gl_from_jbl =joblib.load('optim_gl.jbl')

5.22 s ± 27.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
136 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
gl_from_csv =pd.read_csv('optim_gl.csv', low_memory=False)
display(gl_from_csv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(5), object(79)
memory usage: 211.2+ MB


None

In [17]:
gl_from_jbl =joblib.load('optim_gl.jbl')
display(gl_from_jbl.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: category(78), datetime64[ns](1), float32(77), uint8(5)
memory usage: 79.1 MB


None

# Pandas. Nice but slow

## groupby

https://jacquespeeters.github.io/2017/08/21/parallel-pandas-groupby/

In [18]:
n_jobs = 10
# Create a Dataframe for a minimum example
df = pd.DataFrame()
# 5000 users with approx 100 values
df["user_id"] = np.random.randint(5000, size=500000)
# Generate 500000 random integer values
df["value"] = np.random.randint(30, size=500000)
# Create data_chunk based on modulo of user_id
df["data_chunk"] = df["user_id"].mod(n_jobs* 3)

In [19]:
# Any not optimised and intensive function i want to apply to each group
def group_function(group):
    # Inverse cumulative sum
    group["inv_sum"] = group.iloc[::-1]['value'].cumsum()[::-1].shift(-1).fillna(0)
    return group

In [20]:
%%timeit
normal = df.groupby("user_id").apply(group_function)

6.61 s ± 68.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
def applyParallel(dfGrouped, func, n_treads = n_jobs ):
    with Pool(n_treads) as p:
        ret_list = p.map(func, [group for name, group in dfGrouped])
    return pd.concat(ret_list)

In [22]:
%%timeit
parallel = applyParallel(df.groupby("user_id"), group_function)

3.42 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
def func_group_apply(df):
    return df.groupby("user_id").apply(group_function)

In [24]:
%%timeit
parallel_chunk = applyParallel(df.groupby("data_chunk") , func_group_apply)

1.15 s ± 48.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
normal = df.groupby("user_id").apply(group_function)
parallel = applyParallel(df.groupby("user_id"), group_function)
parallel_chunk = applyParallel(df.groupby("data_chunk") , func_group_apply)
normal = normal.sort_index()
parallel = parallel.sort_index()
parallel_chunk = parallel_chunk.sort_index()

# Check we have same results
print(normal.equals(parallel))
print(normal.equals(parallel_chunk))

True
True


## join vs merge

In [26]:
myids=np.random.choice(np.arange(10000000), size=1000000, replace=False)
df1 = pd.DataFrame(myids, columns=['A'])
df1['B'] = np.random.randint(0,1000,(1000000))
df2 = pd.DataFrame(np.random.permutation(myids), columns=['A2'])
df2['B2'] = np.random.randint(0,1000,(1000000))

In [27]:
%%timeit
    x = df1.merge(df2, how='left', left_on='A', right_on='A2')   

407 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit  
    x = df1.set_index('A').join(df2.set_index('A2'), how='left') 

215 ms ± 3.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%time 
    df1.set_index('A', inplace=True)
    df2.set_index('A2', inplace=True)

CPU times: user 5.28 ms, sys: 32 µs, total: 5.31 ms
Wall time: 3.68 ms


In [30]:
%%timeit
    x = df1.join(df2, how='left')  

52.7 ms ± 345 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
    x = df1.join(df2, how='inner')  

220 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%%timeit
    x = df1.merge(df2, how='left', left_index=True, right_index=True)   

52 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
    x = pd.concat([df1, df2], axis = 1)

197 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Creation of a new variable

link- https://github.com/s-heisler/pycon2017-optimizing-pandas/blob/master/pyCon%20materials/PyCon%20un-sad%20Pandas.ipynb

In [34]:
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

link - https://github.com/s-heisler/pycon2017-optimizing-pandas/blob/master/pyCon%20materials/new_york_hotels.csv

In [35]:
df = pd.read_csv('https://raw.githubusercontent.com/s-heisler/pycon2017-optimizing-pandas/master/pyCon%20materials/new_york_hotels.csv', encoding= 'cp1251')

In [36]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


# Never do that!

### looping function

In [37]:
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [38]:
%%timeit
df['distance'] = haversine_looping(df)

631 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### iteration

In [39]:
%%timeit
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985,\
                                      row['latitude'], row['longitude']))
df['distance'] = haversine_series

173 ms ± 705 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### apply

In [40]:
%%timeit 
df[['distance' ]] = df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

65 ms ± 278 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### vectorization applied on Pandas series

In [41]:
%%timeit 
df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

1.87 ms ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### vectorization applied on NumPy arrays

In [42]:
%%timeit
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

330 µs ± 940 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Cython

In [43]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

In [44]:
%%timeit 
df['distance'] =  haversine_cy(40.671, -73.985, df['latitude'].values, df['longitude'].values)

337 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
from numba import double
from numba.decorators import jit, autojit, njit

In [46]:
haversine_numba = autojit(haversine)

In [47]:
%%timeit 
df['distance'] =  haversine_numba(40.671, -73.985, df['latitude'].values, df['longitude'].values)

379 µs ± 70.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Работает не только с числами

In [48]:
def conc(a, b):
    return a+'_'+b

In [49]:
%%timeit
df['distance' ] = df.apply(lambda row: conc(row['city'], row['state_province']), axis=1)

35.5 ms ± 617 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%%timeit
df[['distance' ]] = conc(df.city, df.state_province)

1.17 ms ± 6.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Не всегда cpython выигрывает у numba


Бывает по-разному

In [51]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef double_every_value_withcpython(x):
    return x*2

In [ ]:
import numba

def double_every_value_nonumba(x):
    return x*2

@njit
def double_every_value_withnumba(x):
    return x*2

auto_numba = autojit(double_every_value_nonumba)


print('double_every_value_nonumba')
%timeit df['col1_doubled'] = df.star_rating.apply(double_every_value_nonumba)

print('just double')
%timeit df['col1_doubled'] = df.star_rating*2
print('double_every_value_withnumba')
%timeit df['col1_doubled'] = double_every_value_withnumba(df.star_rating.values)
print('auto_numba')
%timeit df['col1_doubled'] = auto_numba(df.star_rating.values)
print('double_every_value_withcpython')
%timeit df['col1_doubled'] = double_every_value_withcpython(df.star_rating.values)

double_every_value_nonumba
729 µs ± 2.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
just double
274 µs ± 5.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
double_every_value_withnumba
143 µs ± 777 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
auto_numba
142 µs ± 801 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
double_every_value_withcpython


# Usefull links

* https://www.dataquest.io/blog/pandas-big-data/
* https://jacquespeeters.github.io/2017/08/21/parallel-pandas-groupby/
* https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/
* https://github.com/s-heisler/pycon2017-optimizing-pandas/blob/master/pyCon%20materials/PyCon%20un-sad%20Pandas.ipynb
* http://leadsift.com/loop-map-list-comprehension/
* https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6
* https://pandas.pydata.org/pandas-docs/stable/enhancingperf.html
* http://jakevdp.github.io/blog/2015/02/24/optimizing-python-with-numpy-and-numba/
* https://stackoverflow.com/questions/7165465/optimizing-python-code
* https://wiki.python.org/moin/PythonSpeed/PerformanceTips
* https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/
* http://pypy.org/